In [2]:
pip install tensorflow_datasets

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 35.3 MB/s eta 0:00:00
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21483 sha256=8f8f1616e630deed690d24cf5abe3a6cc61fc01f1eff873250f62940ee06d904
  Stored in directory: /Users/bassel_instructor/Library/Caches/pip/wheels/90/74/b1/9b54c896b8d9409e9268329d4d45ede8a8040abe91c8879932
Successfully built promise
Note: you may need to restart the kernel to use updated packages.


In [1]:
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

## Loading images and labels
(train_ds, train_labels), (test_ds, test_labels) = tfds.load(
    "tf_flowers",
    split=["train[:70%]", "train[:30%]"], ## Train test split
    batch_size=-1,
    as_supervised=True,  # Include labels
)


In [2]:

## Resizing images
train_ds = tf.image.resize(train_ds, (150, 150))
test_ds = tf.image.resize(test_ds, (150, 150))

## Transforming labels to correct format
train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)

In [3]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_ds[0].shape)
base_model.trainable = False ## Not trainable weights

## Preprocessing input
train_ds = preprocess_input(train_ds) 
test_ds = preprocess_input(test_ds)

In [5]:

from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')


model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [6]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)


es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])

Epoch 1/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 62s 943ms/step - accuracy: 0.4431 - loss: 4.0244 - val_accuracy: 0.5642 - val_loss: 1.3057
Epoch 2/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - accuracy: 0.7203 - loss: 0.8113 - val_accuracy: 0.6556 - val_loss: 1.1927
Epoch 3/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 63s 977ms/step - accuracy: 0.8338 - loss: 0.4934 - val_accuracy: 0.6848 - val_loss: 1.0367
Epoch 4/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 71s 1s/step - accuracy: 0.9154 - loss: 0.2590 - val_accuracy: 0.7140 - val_loss: 1.2335
Epoch 5/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 71s 1s/step - accuracy: 0.9306 - loss: 0.2106 - val_accuracy: 0.7276 - val_loss: 1.2037
Epoch 6/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 65s 1s/step - accuracy: 0.9690 - loss: 0.1187 - val_accuracy: 0.7374 - val_loss: 1.2641
Epoch 7/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 62s 960ms/step - accuracy: 0.9847 - loss: 0.0903 - val_accuracy: 0.7237 - val_loss: 1.3442
Epoch 8/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.9935 - loss: 0.0389 - val_accuracy: 0.7198 - 